In [ ]:
# pip install --upgrade langchain
# pip install pypdf
# pip install faiss-cpu

In [215]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain.chat_models import AzureChatOpenAI

In [216]:
model_llm = AzureChatOpenAI(
    temperature = 0.2,
    azure_deployment = 'deployment-text-risk-assessment',
    # azure_endpoint = os.getenv(),
    # openai_api_type = os.getenv()
    openai_api_version = '2023-08-01-preview', 
    openai_api_key = os.getenv('AZURE_OPENAI_API_KEY'),
    openai_api_base = 'https://instance-text-risk-assessment.openai.azure.com/',
)

### Define Tool #1 - BingSearch 

In [217]:
from langchain.utilities import BingSearchAPIWrapper
from langchain.tools.bing_search.tool import BingSearchRun
from langchain.agents import AgentType, initialize_agent, Tool

# os.environ['BING_SUBSCRIPTION_KEY']=os.getenv("BING_SUBSCRIPTION_KEY")
# os.environ['BING_SUBSCRIPTION_KEY']= "28f56faa9d3245648b585190be684fdb"
os.environ['BING_SUBSCRIPTION_KEY']= os.getenv('BING_SUBSCRIPTION_KEY')
os.environ["BING_SEARCH_URL"] = "https://api.bing.microsoft.com/v7.0/search"

searchWrapper = BingSearchAPIWrapper()
tool_BingSearch = BingSearchRun(api_wrapper=searchWrapper)


query_websearch = "what is django"
query_websearch = "what is api gateway"
# result = tool_BingSearch(query_websearch)
# print(result)

#### Build Knowlege Base - Step 1: upload & split documents

In [218]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader

"""
filepath = ('./data/risk_assessment')
loader = PyPDFDirectoryLoader(filepath)

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
documents_chunks = text_splitter.split_documents(documents_original)

documents_chunks

"""

"\nfilepath = ('./data/risk_assessment')\nloader = PyPDFDirectoryLoader(filepath)\n\ndocuments = loader.load()\ntext_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)\ndocuments_chunks = text_splitter.split_documents(documents_original)\n\ndocuments_chunks\n\n"

#### Build Knowledge Base - Step 2: embedding for splitted documents, store text chunks & embeddings into vector db.

In [220]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import OpenAIEmbeddings

model_embedding = OpenAIEmbeddings(
    deployment = 'deployment-embedding-ada-002',
    # model = ,
    chunk_size = 16,
    openai_api_version = '2023-08-01-preview',
    openai_api_base = 'https://instance-text-risk-assessment.openai.azure.com/',
    openai_api_key = os.getenv('AZURE_OPENAI_API_KEY'),
    # openai_api_key = "d7078cc13f3e46138215166baa8bced4",
)

"""
db_vector = FAISS.from_documents(
    docs_chunks,
    embedding = model_embedding,
    
)

"""
retriever = db_vector.as_retriever()

query = 'what is total profit?'
# output = db_vector.similarity_search(query)
# print(output[0].page_content)

# db_vector.save_local("index_faiss_001")
db_vector_new = FAISS.load_local("index_faiss_001", model_embedding)
query = 'any legal issue?' 
# output = db_vector_new.similarity_search(query)
print(output[0].page_content)


ITEM 3. LEGAL PROCEEDINGS
I. SUPPLEMENTAL INFORMATION: We discuss certain legal proceedings in Note 10 to our Consolidated Financial Statements
included in "Item 8. Financial Statements and Supplementary Data," which is captioned "Contingencies," under the sub-caption "Legal
Proceedings." We refer you to that discussion for important information concerning those legal proceedings, including the basis for
such actions and, where known, the relief sought. We provide the following additional information concerning those legal proceedings,
including the name of the lawsuit, the court in which the lawsuit is pending, and the date on which the petition commencing the lawsuit
was filed.
Prescription Opiate Litigation: In re National Prescription Opiate Litigation (MDL No. 2804) (the "MDL"). The MDL is pending in
the U.S. District Court for the Northern District of Ohio and includes over 2,000 cases as of March 3, 2023. The liability phase of a
single, two-county trial in one of the MDL cases 

### Define Tool # 2 - RAG

In [221]:
from langchain.tools.vectorstore.tool import VectorStoreQAWithSourcesTool

tool_RAG = VectorStoreQAWithSourcesTool(
    name = "query_tool_RAG",
    description = "",
    vectorstore = db_vector_new,
    llm = model_llm,
    verbose = True,
)

query = "what is major legal issue?"
# output = tool_RAG(query)
# print(output)

In [55]:
# retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x000001E76A1C7A90>)

#### Define Tool_set

In [222]:
tool_set = [
    Tool.from_function(
        name = "Search from knowledge-base",
        func = tool_RAG.run,
        description = "answer questions based on knowledge-base (i.e. collected documents ) - RAG"
    ),
    Tool.from_function(
        name = "Bing Search",
        func = tool_BingSearch.run,
        description = "answer questions based on web-search"
    ),  
]

### Build Agent - without memory

In [223]:

agent_without_memory = initialize_agent(
    tools = tool_set,
    llm = model_llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True,
    handle_parsing_errors = True,
)

query = "what is walmart's law suit 8 years ago?"
# agent_without_memory.run(query)

### Build Agent - with memory

In [224]:
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor, ZeroShotAgent
from langchain.chains import LLMChain

prefix = """You are a helpful assistant.
"""

suffix = """Begin to generate result. You need to work as a helpful financial assistant. 
{chat_history}
Question: {input}
{agent_scratchpad}
"""

prompt = ZeroShotAgent.create_prompt(
    tools = tool_set,
    prefix = prefix,
    suffix = suffix,
    input_variables = ["input", "chat_history", "agent_scratchpad"],
)

memory = ConversationBufferMemory(
    memory_key = "chat_history",
    
    )

chain_llm = LLMChain(
    llm = model_llm,
    prompt = prompt,
)

agent = ZeroShotAgent(
    llm_chain = chain_llm,
    tools = tool_set,
    verbose = True,
)

agent_with_memory = AgentExecutor.from_agent_and_tools(
    agent = agent,
    tools = tool_set,
    verbose = True,
    memory = memory,
    handle_parsing_errors = True,
)

query = "what is walmarts' law suit 5 years ago?"
# agent_with_memory.run(input = query)

In [225]:
import streamlit as st

st.title('Risk-Assessment Query Tool')
user_web_input = st.text_input('What is the law suit of walmart?')
if st.button('Generate Query Result') and user_web_input:
    output = agent_without_memory.run(user_web_input)
    st.wrtie(output)



### Define Tool - CSV Search

In [104]:
# pip install langchain_experimental

"""

from langchain_experimental.agents.agent_toolkits import create_csv_agent
# from langchain.agents.agent_toolkits import create_csv_agent
# from langchain.agents import create_csv_agent
from langchain.agents import AgentType, initialize_agent

agent_csv = create_csv_agent(
    model_llm,
    './data/csv/titanic.csv',
    verbose = True,
    agent_type = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

query = 'how many row are there?'
agent_csv.run(query)

"""



> Entering new AgentExecutor chain...


Thought: To find out the number of rows in the dataframe, I can use the `shape` attribute of the dataframe which returns a tuple with the number of rows and columns.

Action: python_repl_ast
Action Input: df.shape
Observation: (891, 12)
Thought:I now know the final answer.

Final Answer: There are 891 rows in the dataframe.

> Finished chain.


'There are 891 rows in the dataframe.'

#### Build database: Store .csv files into database 

In [190]:
# pip install sqlite3
import sqlite3
import pandas as pd
"""
df = pd.read_csv("d:/00_data/data_bank_account/card.csv")
df.columns = df.columns.str.strip()
df.to_sql(
    'table_card',
    sqlite3.connect('bank_account_db.db'),
    if_exists = 'replace'
)

df = pd.read_csv("d:/00_data/data_bank_account/order.csv")
df.columns = df.columns.str.strip()
df.to_sql(
    'table_order',
    sqlite3.connect('bank_account_db.db'),
    if_exists = 'replace'
)

df = pd.read_csv("d:/00_data/data_bank_account/accounts.csv")
df.columns = df.columns.str.strip()
df.to_sql(
    'table_accounts',
    sqlite3.connect('bank_account_db.db'),
    if_exists = 'replace'
)

df = pd.read_csv("d:/00_data/data_bank_account/district.csv")
df.columns = df.columns.str.strip()
df.to_sql(
    'table_district',
    sqlite3.connect('bank_account_db.db'),
    if_exists = 'replace'
)

"""

"""
df = pd.read_csv("d:/00_data/data_bank_account/trans.csv")
df.columns = df.columns.str.strip()
df.to_sql(
    'table_trans',
    sqlite3.connect('bank_account_db.db'),
    if_exists = 'replace'
)


df_clients = pd.read_csv("d:/00_data/data_bank_account/clients.csv")
df_clients.columns = df_clients.columns.str.strip()
df_clients.to_sql(
    'table_clients',
    sqlite3.connect('./data/bank_account/bank_account_db.db'),
    if_exists = 'replace'
)

df_loans = pd.read_csv("d:/00_data/data_bank_account/loans.csv")
df_loans.columns = df_loans.columns.str.strip()
df_loans.to_sql(
    'table_loans',
    sqlite3.connect('./data/bank_account/bank_account_db.db'),
    if_exists = 'replace'
)

"""


'\ndf = pd.read_csv("d:/00_data/data_bank_account/trans.csv")\ndf.columns = df.columns.str.strip()\ndf.to_sql(\n    \'table_trans\',\n    sqlite3.connect(\'bank_account_db.db\'),\n    if_exists = \'replace\'\n)\n\n\ndf_clients = pd.read_csv("d:/00_data/data_bank_account/clients.csv")\ndf_clients.columns = df_clients.columns.str.strip()\ndf_clients.to_sql(\n    \'table_clients\',\n    sqlite3.connect(\'./data/bank_account/bank_account_db.db\'),\n    if_exists = \'replace\'\n)\n\ndf_loans = pd.read_csv("d:/00_data/data_bank_account/loans.csv")\ndf_loans.columns = df_loans.columns.str.strip()\ndf_loans.to_sql(\n    \'table_loans\',\n    sqlite3.connect(\'./data/bank_account/bank_account_db.db\'),\n    if_exists = \'replace\'\n)\n\n'

#### Build database: Test database for the existance of dataset 

In [ ]:
"""

curr = sqlite3.connect('./data/bank_account/bank_account_db.db').cursor()

curr.execute('''SELECT * from table_clients''')
for record in curr.fetchall():
    print(record)

"""

### Define Tool - SQL query 

In [195]:
"""

from langchain.sql_database import SQLDatabase
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

db = SQLDatabase.from_uri("sqlite:///./data/bank_account/bank_account_db.db")
tool_sql = SQLDatabaseToolkit(
    db = db,
    llm = model_llm,
)

agent = create_sql_agent(
    llm = model_llm,
    toolkit = tool_sql,
    agent_type = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True,
)

query = ("how many rows are there in the accounts table?")
output = agent.run(query)
print(output)

"""





> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: table_accounts, table_clients, table_disp, table_district, table_loans, table_loans_2
Thought:The table that seems to hold account information is "table_accounts". I will now check the schema of this table to ensure I query the correct columns.
Action: sql_db_schema
Action Input: table_accounts
Observation: 
CREATE TABLE table_accounts (
	"index" INTEGER, 
	"Unnamed: 0" INTEGER, 
	account_id INTEGER, 
	district_id INTEGER, 
	frequency TEXT, 
	date TEXT
)

/*
3 rows from table_accounts table:
index	Unnamed: 0	account_id	district_id	frequency	date
0	0	576	55	settle_monthly	1993-01-01
1	1	3818	74	settle_monthly	1993-01-01
2	2	704	55	settle_monthly	1993-01-01
*/
Thought:I now know the structure of the table_accounts table and can write a query to count the number of rows in it.
Action: sql_db_query_checker
Action Input: SELECT COUNT(*) FROM table_accounts;
Observation: The provided SQLite query d

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2023-08-01-preview have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..


The query to count the rows in the table_accounts table is correct. I can now execute it to get the result.
Action: sql_db_query
Action Input: SELECT COUNT(*) FROM table_accounts;
Observation: [(4500,)]
Thought:I now know the final answer
Final Answer: There are 4500 rows in the accounts table.

> Finished chain.
There are 4500 rows in the accounts table.


In [89]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

template = """Answer the question based only on the following context: 
{context}

Question:{question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model_llm
    | StrOutputParser()
)

chain.invoke("what is the profit")

'Based on the provided context, the profit referred to is likely the "Consolidated net income" for the fiscal years mentioned in the document. The consolidated net income figures presented are as follows:\n\n- For fiscal year 2023: $11,292 million\n- For fiscal year 2022: $13,940 million\n- For fiscal year 2021: $13,706 million\n\nThese figures represent the net income after accounting for all expenses, including operating expenses, interest, taxes, and other gains and losses.'

In [ ]:
# pip install watermark

%load_ext watermark
%watermark -a "Sudarshan Koirala" -vmp langchain,openai

In [ ]:
# pip install sqlalchemy
"""
import sqlalchemy
df = pd.read_sql_table(
    "table_loans", 
    sqlalchemy.create_engine("sqlite:///./data/bank_account/bank_account_db.db"),
    )
"""

In [ ]:
"""
from langchain.chains import create_sql_query_chain
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

db_sql = SQLDatabase.from_uri("sqlite:///./data/bank_account/card.db")

chain_sql = create_sql_query_chain(model_llm, db_sql)

prompt = {"question":"how many rows are there?"}
output = chain.invoke(prompt)

"""
# print(output)